# Replace Secured Views in Shares with Dynamic Tables

## Overview

This notebook is designed to streamline the management and utilization of secured views within Snowflake shares by converting them into dynamic tables. The process helps in enhancing data freshness, enabling more efficient data management practices, and optimizing query performance.

## Process

1. **Identify Secured Views in Database**: 

    The first step involves scanning the Snowflake database to identify all secured views. Secured views are those that have restricted access, ensuring that sensitive data is protected and only accessible to authorized users.

    We query the `SNOWFLAKE.ACCOUNT_USAGE.VIEWS` system view to retrieve a list of all views in the account, filtering specifically for those marked as secured. This helps in segregating the views that are crucial for sensitive data handling.
  
2. **Extract Object Dependencies**:

    Once the secured views are identified, the next step is to extract their dependencies. This includes understanding which tables, functions, or other database objects the views depend on.

    This involves querying the `SNOWFLAKE.ACCOUNT_USAGE.OBJECT_DEPENDENCIES` system function to map out all underlying objects that the secured views interact with. This mapping is essential for ensuring that all dependent objects are considered when converting the views into dynamic tables.

3. **Qualify Secured Views for conversion**: 

    Not all secured views may be suitable for conversion to dynamic tables. This step evaluates each secured view to determine if it meets the criteria for conversion based on factors such as complexity, dependencies, and potential performance benefits.

    Apply a set of predefined rules or checks to assess which views are ideal candidates for conversion. This might include checking for views that do not involve overly complex SQL operations or those that are frequently accessed and would benefit most from dynamic table features.

4. **Create conversion Code**: 

    For views that qualify, generate the SQL code needed to convert these secured views into dynamic tables. Dynamic tables in Snowflake allow for automatic data management features such as clustering and micro-partitioning, which improve query performance and data retrieval speed.

![process_flow](https://lh3.googleusercontent.com/d/1hQMggZQ706k9FSApWXMdFMCm4TIx1-yl)

# Initialize Session

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import re

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

#tag session
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"dt_secure_view_conversion_task","version":{{"major":1, "minor":0}},"attributes":"session_tag"}}'""").collect()

st.success(f"Session initialized 🎉")


# Functions used in Notebook

In [ ]:

def get_shares(session):
    """
    Retrieves all shares from Snowflake.
    
    :param session: Snowflake session object
    :return: Pandas DataFrame of shares
    """
    shares_query = "SHOW SHARES;"
    shares_rows = session.sql(shares_query).collect()
    df_shares = pd.DataFrame(shares_rows)
    return df_shares


def filter_shares_by_database(df_shares, database_name):
    """
    Filters the shares to only include those related to the given database.
    
    :param df_shares: DataFrame containing all shares
    :param database_name: Name of the database to filter on
    :return: Filtered DataFrame of shares related to the database
    """
    return df_shares[df_shares['database_name'] == database_name.upper()]


def get_secure_views(session, database_name):
    """
    Retrieves all secure views from the given database along with their view definitions.
    
    :param session: Snowflake session object
    :param database_name: Name of the database
    :return: Pandas DataFrame of secure views with their definitions
    """
    secure_views_query = f"""
        SELECT TABLE_SCHEMA, TABLE_NAME, VIEW_DEFINITION
        FROM SNOWFLAKE.ACCOUNT_USAGE.VIEWS
        WHERE TABLE_CATALOG = '{database_name}'
        AND DELETED IS NULL
        and IS_SECURE = 'YES';
    """
    return session.sql(secure_views_query).to_pandas()


def get_shared_views_for_share(session, share_name):
    """
    Retrieves all shared views for a given share.
    
    :param session: Snowflake session object
    :param share_name: Name of the share
    :return: List of tuples containing schema and view names, or an empty list if no views are found
    """
    desc_share_query = f"DESC SHARE {share_name};"
    share_details_rows = session.sql(desc_share_query).collect()
    
    # If no rows are returned, return an empty list
    if not share_details_rows:
        return []

    df_share_details = pd.DataFrame(share_details_rows)
    
    # Filter for views and split the 'name' column into database, schema, and view
    df_share_views = df_share_details[df_share_details['kind'] == 'VIEW'].copy()

    # If no views are found, return an empty list
    if df_share_views.empty:
        return []
    
    df_share_views[['DATABASE', 'SCHEMA', 'VIEW']] = df_share_views['name'].str.split('.', expand=True)
    
    return df_share_views[['SCHEMA', 'VIEW']].values.tolist()



def get_all_shared_views(session, df_shares_filtered):
    """
    Loops through all shares and retrieves shared views, including the share name.
    
    :param session: Snowflake session object
    :param df_shares_filtered: Filtered DataFrame of shares related to the database
    :return: List of tuples containing share name, schema name, and view name for all shares
    """
    shared_views = []
    for share in df_shares_filtered['name']:
        # Get shared views for the current share
        views_for_share = get_shared_views_for_share(session, share)
        
        # Add the share name to each entry in the result
        for schema_name, view_name in views_for_share:
            shared_views.append((share, schema_name, view_name))
    
    return shared_views



def check_if_views_are_shared(df_secure_views, shared_views):
    """
    Adds columns to df_secure_views indicating if the view is part of any shared views and the corresponding share name.
    
    :param df_secure_views: DataFrame of secure views
    :param shared_views: List of tuples containing share name, schema name, and view name
    :return: DataFrame with 'IS_SHARED' and 'SHARE_NAME' columns added
    """
    # Convert shared_views to a set of tuples for efficient lookups
    shared_views_dict = { (schema_name, view_name): share_name for share_name, schema_name, view_name in shared_views }
    
    def get_share_info(row):
        # Check if the (schema, view) exists in the shared_views dictionary and return the share name
        schema_view_tuple = (row['TABLE_SCHEMA'], row['TABLE_NAME'])
        if schema_view_tuple in shared_views_dict:
            return True, shared_views_dict[schema_view_tuple]
        return False, None

    # Apply the function to add 'IS_SHARED' and 'SHARE_NAME' columns
    df_secure_views[['IS_SHARED', 'SHARE_NAME']] = df_secure_views.apply(
        lambda row: pd.Series(get_share_info(row)),
        axis=1
    )

    return df_secure_views


def get_ddl_for_shared_views(session, df_secure_views):
    """
    Loops through each secure view that is being shared and retrieves the DDL for it.
    
    :param session: Snowflake session object
    :param df_secure_views: DataFrame containing secure views with the 'IS_SHARED' column
    :return: Dictionary where keys are view names and values are their DDL statements
    """
    ddl_statements = {}

    # Filter only the secure views that are being shared
    shared_views = df_secure_views[df_secure_views['IS_SHARED'] == True]

    # Loop through each shared view and get its DDL
    for index, row in shared_views.iterrows():
        schema = row['TABLE_SCHEMA']
        view_name = row['TABLE_NAME']
        
        # Use GET_DDL function to get the DDL for the view
        get_ddl_query = f"SELECT GET_DDL('VIEW', '{schema}.{view_name}');"
        
        # Execute the query and fetch the DDL
        ddl_result = session.sql(get_ddl_query).collect()
        ddl_statements[f"{schema}.{view_name}"] = ddl_result[0][0]
    
    return ddl_statements    

def check_object_type(session, database_name, schema_name, object_name):
    """
    Checks whether a given object is a table or a view in Snowflake.
    
    :param session: Snowflake session object
    :param database_name: Name of the database
    :param schema_name: Name of the schema
    :param object_name: Name of the object (table/view)
    :return: String indicating whether the object is a 'TABLE', 'VIEW', or 'UNKNOWN'
    """
    # Check if the object is a table
    table_query = f"""
        SELECT COUNT(*) FROM {database_name}.INFORMATION_SCHEMA.TABLES
        WHERE TABLE_SCHEMA = '{schema_name}' AND TABLE_NAME = '{object_name}';
    """
    table_result = session.sql(table_query).collect()
    
    if table_result[0][0] > 0:
        return 'TABLE'
    
    # Check if the object is a view
    view_query = f"""
        SELECT COUNT(*) FROM {database_name}.INFORMATION_SCHEMA.VIEWS
        WHERE TABLE_SCHEMA = '{schema_name}' AND TABLE_NAME = '{object_name}';
    """
    view_result = session.sql(view_query).collect()
    
    if view_result[0][0] > 0:
        return 'VIEW'
    
    # If neither, return 'UNKNOWN'
    return 'UNKNOWN'


def analyze_dependencies_with_cluster_keys(session, ddl_dict):
    """
    Analyzes the dependencies of each view in the DDL dictionary to determine whether the dependencies are tables or views,
    and if a table, also checks for clustering keys.
    
    :param session: Snowflake session object
    :param ddl_dict: Dictionary where keys are view names and values are their DDL statements
    :return: Dictionary where keys are view names and values are lists of tuples (dependency, type, cluster_keys)
    """
    view_analysis = {}
    
    # Loop through each view in the dictionary
    for view_name, ddl_statement in ddl_dict.items():
        # Extract the dependencies
        dependencies = get_object_dependencies(ddl_statement)
        
        # Initialize a list to hold the analyzed dependencies for this view
        analyzed_dependencies = []
        
        # Analyze each dependency
        for dep in dependencies:
            # Split the fully qualified name (e.g., SCHEMA.OBJECT)
            parts = dep.split('.')
            
            if len(parts) == 3:  # Ensure the object is fully qualified (database.schema.object)
                database_name, schema_name, object_name = parts
                # Check if it's a table or a view
                object_type = check_object_type(session, database_name, schema_name, object_name)
                
                # Initialize an empty list for cluster keys
                cluster_keys = []
                
                # If it's a table, retrieve the cluster keys
                if object_type == 'TABLE':
                    cluster_keys = get_cluster_keys(session, database_name, schema_name, object_name)
                
                # Append the dependency along with its type and cluster keys
                analyzed_dependencies.append((dep, object_type, cluster_keys))
        
        # Store the result in the view_analysis dictionary
        view_analysis[view_name] = analyzed_dependencies
    
    return view_analysis

def get_cluster_keys(session, database_name, schema_name, table_name):
    """
    Retrieves the cluster keys for a given table in Snowflake by using the SHOW TABLES LIKE command.
    
    :param session: Snowflake session object
    :param database_name: Name of the database
    :param schema_name: Name of the schema
    :param table_name: Name of the table
    :return: List of cluster keys, if present
    """
    # Query to show information about the table (including clustering keys)
    show_table_query = f"""
        SHOW TABLES LIKE '{table_name}' IN SCHEMA {database_name}.{schema_name};
    """
    
    # Execute the query and collect the results
    table_info_result = session.sql(show_table_query).collect()

    # Check if the table has clustering keys in the 'cluster_by' column
    if table_info_result:
        cluster_by_value = table_info_result[0]['cluster_by']
        if cluster_by_value:
            # Return the clustering keys as a list (split by commas if there are multiple keys)
            return [key.strip() for key in cluster_by_value.replace('LINEAR(', '').replace(')', '').split(',')]
    
    return []  # Return an empty list if no clustering keys are found
    
def get_dynamic_table_ddl_for_views(view_definitions, dependencies_dict, database_name, target_lag, refresh_mode):
    """
    Constructs the DDL for dynamic tables based on shared secure views, their dependencies, and clustering keys.
    Drops the original views before creating the dynamic table and adds the dynamic table to the share.
    
    :param view_definitions: DataFrame containing view names, view definitions, and share information.
    :param dependencies_dict: Dictionary where keys are view names and values are lists of dependency details.
    :param database_name: The database name where dynamic tables will be created.
    :param target_lag: The target lag for the dynamic table.
    :param refresh_mode: The refresh mode for the dynamic table.
    :return: Dictionary where keys are view names and values are the generated DDL statements for dynamic tables.
    """
    dynamic_table_ddls = {}

    for index, row in view_definitions.iterrows():
        view_name = row['TABLE_NAME']
        view_ddl = row['VIEW_DEFINITION']
        share_name = row['SHARE_NAME']  # Assuming this column contains the share name

        # Extract only the SELECT statement from the view DDL
        select_start_idx = view_ddl.lower().find("select")
        if select_start_idx == -1:
            print(f"Skipping view {view_name}: No SELECT statement found in DDL.")
            continue
        select_statement = view_ddl[select_start_idx:].strip()

        # Initialize clustering clause
        cluster_by_clause = ""

        # Check if there are dependencies for this view and create CLUSTER BY clause
        if view_name in dependencies_dict:
            cluster_keys = [dep['REFERENCED_OBJECT_NAME'] for dep in dependencies_dict[view_name] if dep['REFERENCED_OBJECT_DOMAIN'] == 'TABLE']
            if cluster_keys:
                cluster_by_clause = f" CLUSTER BY ({', '.join(cluster_keys)})"

        # Generate the SQL for dropping the view if it exists
        drop_view_sql = f"DROP VIEW IF EXISTS {database_name}.{view_name};"

        # Generate the SQL for creating the dynamic table using the SELECT statement and cluster keys
        create_table_sql = f"""
        CREATE OR REPLACE DYNAMIC TABLE {database_name}.{view_name}
        COMMENT = '{{"origin":"sf_sit","name":"dt_secure_view_conversion_task,"version":{{"major":1, "minor":1}},"attributes":{{"table":"{view_name}", "type":"dynamictable"}}}}'
        REFRESH_MODE = {refresh_mode}
        TARGET_LAG = '{target_lag}'
        WAREHOUSE = 'COHORT_BUILDER_LOAD_WH'
        {cluster_by_clause}
        AS
        {select_statement}
        """

        # Generate the SQL for adding the dynamic table to the share
        add_to_share_sql = f"ALTER SHARE {share_name} ADD TABLE {database_name}.{view_name};"

        # Combine all the SQL into a single script for this view
        full_ddl_script = f"""
        {drop_view_sql}
        
        {create_table_sql.strip()}
        
        {add_to_share_sql}
        """

        # Add the generated DDL to the dictionary
        dynamic_table_ddls[view_name] = full_ddl_script.strip()

    return dynamic_table_ddls


def remove_comments(sql_code):
    """
    Removes SQL comments (both single-line and multi-line) from the given SQL code.
    
    :param sql_code: The SQL code as a string.
    :return: The SQL code with comments removed.
    """
    # Remove single-line comments (starting with --)
    sql_code = re.sub(r'--.*', '', sql_code)
    
    # Remove multi-line comments (starting with /* and ending with */)
    sql_code = re.sub(r'/\*.*?\*/', '', sql_code, flags=re.DOTALL)
    
    return sql_code


def extract_query_from_view_ddl(view_ddl):
    """
    Extracts the query (including CTEs if present) from a view's DDL.
    
    :param view_ddl: The DDL statement of the view as a string
    :return: The query as a string (can start with SELECT or WITH) or None if no valid query is found
    """
    # Ensure the DDL is in lowercase to make it case-insensitive
    ddl_lower = view_ddl.lower()

    # Find the starting index of the SELECT or WITH statement
    select_start_idx = ddl_lower.find("select")
    with_start_idx = ddl_lower.find("with")

    # Determine which comes first, SELECT or WITH, if either exists
    if select_start_idx == -1 and with_start_idx == -1:
        print("No SELECT or WITH statement found in the view DDL.")
        return None

    # Use the position of the first occurrence of either SELECT or WITH
    query_start_idx = min(
        idx for idx in [select_start_idx, with_start_idx] if idx != -1
    )

    # Extract everything starting from the SELECT or WITH statement
    query_statement = view_ddl[query_start_idx:].strip()

    # Optionally, remove any trailing semicolons if they exist
    if query_statement.endswith(";"):
        query_statement = query_statement[:-1]

    return query_statement





def create_dependency_dict(session, shared_views, database_name):
    """
    Creates a clean dependency dictionary for each shared view, containing 
    REFERENCED_OBJECT_DOMAIN, REFERENCED_OBJECT_NAME, along with schema and 
    catalog information.
    
    :param session: Snowflake session object
    :param shared_views: Pandas DataFrame containing shared views
    :return: Dictionary with view names as keys, and references with schema and 
             catalog information
    """
    dependency_dict = {}

    # Loop through each row in the shared_views DataFrame
    for index, row in shared_views.iterrows():
        view_name = row['TABLE_NAME']
        schema_name = row['TABLE_SCHEMA']
        database_name = database_name

        # Query to find dependencies for each shared view, selecting the required columns
        dependency_query = f"""
        SELECT REFERENCED_OBJECT_DOMAIN, REFERENCED_OBJECT_NAME, REFERENCED_SCHEMA, REFERENCED_DATABASE
        FROM SNOWFLAKE.ACCOUNT_USAGE.OBJECT_DEPENDENCIES
        WHERE REFERENCING_OBJECT_NAME = '{view_name}'
        AND REFERENCING_SCHEMA = '{schema_name}'
        AND REFERENCING_DATABASE = '{database_name}';
        """
        
        # Execute the query and convert results to a DataFrame
        dependency_df = session.sql(dependency_query).to_pandas()

        # Convert the result to a list of dictionaries and add schema, database info
        dependencies = dependency_df.to_dict('records')

        # Store the dependencies in the dictionary with the view name as the key
        dependency_dict[view_name] = dependencies
    
    return dependency_dict

def get_clustering_keys(session, table_name, schema_name, database_name):
    """
    Retrieves the clustering key information for a given table.
    
    :param session: Snowflake session object
    :param table_name: Name of the table
    :param schema_name: Name of the schema
    :param database_name: Name of the database
    :return: Clustering key or None if no clustering key exists
    """
    # Query to get the clustering key from the INFORMATION_SCHEMA for the given table
    clustering_key_query = f"""
    SELECT CLUSTERING_KEY
    FROM {database_name}.INFORMATION_SCHEMA.TABLES
    WHERE TABLE_NAME = '{table_name}'
    AND TABLE_SCHEMA = '{schema_name}';
    """
    
    # Execute the query and fetch the result
    clustering_key_df = session.sql(clustering_key_query).to_pandas()

    if not clustering_key_df.empty:
        return clustering_key_df['CLUSTERING_KEY'].iloc[0]
    else:
        return None

# Function to clean up the clustering key
def clean_clustering_key(clustering_key):
    if pd.isna(clustering_key):
        return None
    # Use regex to extract the content inside parentheses
    match = re.search(r'\((.*?)\)', clustering_key)
    if match:
        return match.group(1)  # Return the key inside parentheses
    return clustering_key

def check_dynamic_table_type(session, df_secure_views_with_shared_info, database_name):
    """
    Creates dynamic tables for the views in df_secure_views_with_shared_info and retrieves the refresh mode for each table.
    
    :param session: Snowflake session object
    :param df_secure_views_with_shared_info: DataFrame of view names and their shared information
    :param database_name: Name of the database where dynamic tables will be created
    
    :return: DataFrame containing view names and their corresponding refresh_mode values
    """
    results = []

    # Iterate through each row in the DataFrame
    for index, row in df_secure_views_with_shared_info.iterrows():
        view_name = row['TABLE_NAME']
        schema_name = row['TABLE_SCHEMA']
        view_ddl = row['VIEW_DEFINITION']  # Extract the DDL directly from the DataFrame
        
        if not view_ddl:
            print(f"Skipping view {view_name}: No DDL found.")
            continue

        # Extract the select statement from the DDL
        select_statement = extract_query_from_view_ddl(view_ddl)

        # Create the dynamic table SQL
        create_table_sql = f"""
        CREATE OR REPLACE DYNAMIC TABLE {database_name}.{schema_name}.test_dynamic_table
        COMMENT = '{{"origin":"sf_sit","name":"dt_secure_view_conversion_task","version":{{"major":1, "minor":1}},"attributes":{{"table":"{view_name}", "type":"dynamictable"}}}}'
        TARGET_LAG = DOWNSTREAM 
        WAREHOUSE = 'COHORT_BUILDER_LOAD_WH'
        AS
        {select_statement};
        """
        # Execute the create table SQL
        session.sql(create_table_sql).collect()

        # Query to check the refresh mode
        check_refresh_mode_query = f"SHOW DYNAMIC TABLES LIKE 'test_dynamic_table' IN SCHEMA {database_name}.{schema_name};"
        check_refresh_mode = session.sql(check_refresh_mode_query).collect()

        # Extract the refresh_mode and refresh_mode_reason column values
        if check_refresh_mode:
            refresh_mode_value = check_refresh_mode[0]['refresh_mode']
            refresh_mode_reason = check_refresh_mode[0]['refresh_mode_reason']
        else:
            refresh_mode_value = None
            refresh_mode_reason = None

        # Store the result as a tuple of view name and refresh mode
        results.append((view_name, refresh_mode_value, refresh_mode_reason))

        # Drop the dynamic table after checking
        session.sql(f"DROP DYNAMIC TABLE {database_name}.{schema_name}.test_dynamic_table").collect()

    # Convert the results to a DataFrame
    df_results = pd.DataFrame(results, columns=["view_name", "refresh_mode", "refresh_mode_reason"])

    return df_results



# 1: Identify Secured Views in Database

In this section, you will select the **database** from which you would like to convert the secured views to dynamic tables. 

Use the following code cell to select the database for migration. It will browse through the database and return all the secure views available.


In [ ]:
database_name = 'MARKETING_DATA'

df_shares = get_shares(session)
    
# Step 2: Filter shares related to the selected database
df_shares_filtered = filter_shares_by_database(df_shares, database_name)

# Step 3: Get all secure views in the selected database
df_secure_views = get_secure_views(session, database_name)

# Step 4: Get a list of views that are a part of shares in this database
shared_views = get_all_shared_views(session, df_shares_filtered)

# Step 5: Get a list of views that are secure and shared.
df_secure_views_with_shared_info = check_if_views_are_shared(df_secure_views, shared_views)

df_secure_views_with_shared_info

# 2: Extract Object Dependencies

This code performs the following tasks:

1. **Dependency Dictionary Creation**: 
   - The `create_dependency_dict` function creates a dictionary of dependencies for each shared view in a Snowflake database, querying the `SNOWFLAKE.ACCOUNT_USAGE.OBJECT_DEPENDENCIES` table. It extracts referenced object domain, name, schema, and catalog information for each view.

Finally, the `dependency_analysis` stores the output of the shared view dependency analysis.


In [ ]:
ddl_statements = {}
view_dependencies = {}

shared_views = df_secure_views_with_shared_info[df_secure_views['IS_SHARED'] == True]

dependency_analysis = create_dependency_dict(session, shared_views, database_name)

dependency_analysis


### Building a DataFrame with View Dependencies and Clustering Keys

This code processes the dependency analysis from Snowflake and constructs a detailed DataFrame containing information about each view's referenced objects, including clustering keys when applicable. The process involves the following steps:

1. **DataFrame Initialization**:
   - A new DataFrame `dependency_with_clusters_df` is created with columns: `VIEW_NAME`, `REFERENCED_OBJECT_DOMAIN`, `REFERENCED_OBJECT_NAME`, `REFERENCED_SCHEMA`, `REFERENCED_DATABASE`, and `CLUSTERING_KEY`.

2. **Iterating Through Dependencies**:
   - For each view in `dependency_analysis`, the referenced objects (like tables, schemas, databases) are extracted and stored.
   
3. **Clustering Key Retrieval**:
   - For each referenced object of domain type `'TABLE'`, the clustering key is retrieved by querying the `INFORMATION_SCHEMA.TABLES` table of the corresponding database.
   
4. **Appending Results to DataFrame**:
   - A new row is created for each referenced object with its corresponding clustering key (if available) and appended to the DataFrame using `pd.concat()`.

The final DataFrame (`dependency_with_clusters_df`) consolidates the view dependencies along with the clustering keys, providing a comprehensive reference for further analysis.


In [ ]:
columns = ['VIEW_NAME', 'REFERENCED_OBJECT_DOMAIN', 'REFERENCED_OBJECT_NAME', 'REFERENCED_SCHEMA', 'REFERENCED_DATABASE', 'CLUSTERING_KEY']
dependency_with_clusters_df = pd.DataFrame(columns=columns)

# Loop through the dictionary
for view_name, dependencies in dependency_analysis.items():
    for dependency in dependencies:
        referenced_object_name = dependency['REFERENCED_OBJECT_NAME']
        referenced_schema = dependency['REFERENCED_SCHEMA']
        referenced_database = dependency['REFERENCED_DATABASE']
        referenced_domain = dependency['REFERENCED_OBJECT_DOMAIN']
        
        clustering_key = None

        # If the domain is 'TABLE', fetch the clustering key
        if referenced_domain == 'TABLE':
            # Query to get the clustering key (assuming you have a session object and function for this)
            clustering_key_query = f"""
            SELECT CLUSTERING_KEY
            FROM {referenced_database}.INFORMATION_SCHEMA.TABLES
            WHERE TABLE_NAME = '{referenced_object_name}'
            AND TABLE_SCHEMA = '{referenced_schema}';
            """
            
            # Execute the query to retrieve the clustering key
            clustering_key_df = session.sql(clustering_key_query).to_pandas()
            
            if not clustering_key_df.empty:
                clustering_key = clustering_key_df['CLUSTERING_KEY'].iloc[0]
        
        # Create a new DataFrame row to append
        new_row = pd.DataFrame([{
            'VIEW_NAME': view_name,
            'REFERENCED_OBJECT_DOMAIN': referenced_domain,
            'REFERENCED_OBJECT_NAME': referenced_object_name,
            'REFERENCED_SCHEMA': referenced_schema,
            'REFERENCED_DATABASE': referenced_database,
            'CLUSTERING_KEY': clustering_key
        }])
        
        # Use pd.concat() to append the new row to the DataFrame
        dependency_with_clusters_df = pd.concat([dependency_with_clusters_df, new_row], ignore_index=True)

dependency_with_clusters_df['CLUSTERING_KEY'] = dependency_with_clusters_df['CLUSTERING_KEY'].apply(clean_clustering_key)
dependency_with_clusters_df


# 3: Qualify Secured Views for conversion

### Checking Dynamic Table Type and Refresh Mode in Snowflake

This code defines the `check_dynamic_table_type` function, which performs the following tasks:

1. **Purpose**:
   - The function checks the refresh mode of dynamic tables created from secure views in Snowflake. It processes each view from the `df_secure_views_with_shared_info` DataFrame, creates a temporary dynamic table, retrieves the `refresh_mode`, and then drops the table.

2. **Steps Involved**:
   - **View Iteration**: Iterates over the rows of `df_secure_views_with_shared_info` to extract the view name, schema, and DDL (View Definition).
   - **Select Statement Extraction**: Extracts the `SELECT` query from the view's DDL.
   - **Dynamic Table Creation**: Creates a temporary dynamic table in Snowflake using the `CREATE OR REPLACE DYNAMIC TABLE` command.
   - **Refresh Mode Check**: Queries the `SHOW DYNAMIC TABLES` command to check the `refresh_mode` and `refresh_mode_reason` for the created dynamic table.
   - **Table Dropping**: Drops the dynamic table after retrieving the necessary information.

3. **Output**:
   - Returns a DataFrame containing the view names, their associated `refresh_mode`, and the reason for the refresh mode (`refresh_mode_reason`).

This function helps to verify how dynamic tables created from specific views behave with respect to refresh mechanisms in Snowflake.


In [ ]:
check_dynamic_table_type(session, df_secure_views_with_shared_info, database_name)


### Select Secured Views for convertions

Add the names of the secured views you want to filter out from the dynamic table conversion ddl that is generated in the next step

In [ ]:
filter_list = [] 
#['ANALYTICS.VIEW_FAN_PROFILE_NONUS_TICKET_PURCHASERS']

filtered_dict = {key: value for key, value in dependency_analysis.items() if key not in filter_list}

filtered_dict.keys()


## Dynamic table DDL

Please follow the steps below to run the function that generates dynamic tables. Before proceeding, ensure that all necessary clusters are reviewed. This will help optimize performance and avoid potential resource bottlenecks.

### Important Notes:

**Full Refresh Mode:** For any dynamic table where a full refresh is required, the function has automatically set the `REFRESH_MODE` to `FULL`. This decision is based on the complexity of the underlying query.

**Overriding the Refresh Mode:**
    If you would like to use an incremental refresh instead of a full refresh, you can override this behavior by re-creating the dynamic table with `REFRESH_MODE=INCREMENTAL`.
    We recommend thoroughly reviewing Snowflake's guidelines on Dynamic Table Performance before making this adjustment.

In [ ]:
get_dynamic_table_ddl_for_views(df_secure_views_with_shared_info,filtered_dict, database_name, target_lag = '10 minutes', refresh_mode = 'AUTO')